In [1]:
import os
cwd = os.getcwd()

In [2]:
cd {cwd}

/mnt/File/shamgholi/projects/person_reid/AP3D


# load model

In [3]:
import sys
import os
sys.path.append(cwd)
import time
import math
import os.path as osp
import numpy as np

import torch
from torch.utils.data import DataLoader

from config import Config
import models
import transforms.spatial_transforms as ST
import transforms.temporal_transforms as TT
import tools.data_manager as data_manager
from tools.video_loader import VideoDataset
from tools.utils import Logger
from tools.eval_metrics import evaluate
from commons import modify_model
from tqdm import tqdm
import random


def seed_everythings():
    seed = 1
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)



class Args:
    #"/home/hadi/iust/datasets"
    #"/home/shamgholi/iust/datasets" 
    #"/mnt/File/shamgholi/datasets"
    root = "/mnt/File/shamgholi/datasets"
    dataset = "mars"
    workers = 0
    height = 256
    width = 128
    # test_frames = 8
    arch = "ap3dres50"
    resume = "./"
    pretrain = os.path.join(cwd, "logs/row41/best_model.pth.tar")
    test_epochs = [240]
    distance = "cosine" #cosine
    gpu = "0"
    test_batch = 6
    

/mnt/1tra/shamgholi/miniconda3/envs/reid/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Default Test (Eval)

In [4]:
def test(model, queryloader, galleryloader, use_gpu, args):
    # test using 4 frames
    since = time.time()
    model.eval()

    qf, q_pids, q_camids = [], [], []
    for batch_idx, (vids, pids, camids) in enumerate(queryloader):
        if use_gpu:
            vids = vids.cuda()

        feat = model(vids)
        feat = feat.mean(1)
        feat = model.bn(feat)
        feat = feat.data.cpu()

        qf.append(feat)
        q_pids.extend(pids)
        q_camids.extend(camids)

    qf = torch.cat(qf, 0)
    q_pids = np.asarray(q_pids)
    q_camids = np.asarray(q_camids)
    print("Extracted features for query set, obtained {} matrix".format(qf.shape))

    gf, g_pids, g_camids = [], [], []
    for batch_idx, (vids, pids, camids) in enumerate(galleryloader):
        if use_gpu:
            vids = vids.cuda()

        feat = model(vids)
        feat = feat.mean(1)
        feat = model.bn(feat)
        feat = feat.data.cpu()

        gf.append(feat)
        g_pids.extend(pids)
        g_camids.extend(camids)

    gf = torch.cat(gf, 0)
    g_pids = np.asarray(g_pids)
    g_camids = np.asarray(g_camids)

    if args.dataset == 'mars':
        # gallery set must contain query set, otherwise 140 query imgs will not have ground truth.
        gf = torch.cat((qf, gf), 0)
        g_pids = np.append(q_pids, g_pids)
        g_camids = np.append(q_camids, g_camids)

    print("Extracted features for gallery set, obtained {} matrix".format(gf.shape))

    time_elapsed = time.time() - since
    print('Extracting features complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print("Computing distance matrix")
    m, n = qf.size(0), gf.size(0)
    distmat = torch.zeros((m,n))

    if args.distance == 'euclidean':
        distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                  torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
        for i in range(m):
            distmat[i:i+1].addmm_(1, -2, qf[i:i+1], gf.t())
    else:
        q_norm = torch.norm(qf, p=2, dim=1, keepdim=True)
        g_norm = torch.norm(gf, p=2, dim=1, keepdim=True)
        qf = qf.div(q_norm.expand_as(qf))
        gf = gf.div(g_norm.expand_as(gf))
        for i in range(m):
            distmat[i] = - torch.mm(qf[i:i+1], gf.t())
    distmat = distmat.numpy()
    
    print("Computing CMC and mAP")
    cmc, mAP = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)

    print("Results ----------")
    print('top1:{:.1%} top5:{:.1%} top10:{:.1%} mAP:{:.1%}'.format(cmc[0],cmc[4],cmc[9],mAP))
    print("------------------")

    return cmc[0]




def experiment():
    print("Initializing model: {}".format(args.arch))
    seed_everythings()
    model = models.init_model(name=args.arch, conf=conf, num_classes=dataset.num_train_pids)
    print("Model size: {:.5f}M".format(sum(p.numel() for p in model.parameters())/1000000.0))
    model = model.to(device)
    model = model.eval()
    modify_model(model, args, conf)
    with torch.no_grad():
        test(model, queryloader, galleryloader, use_gpu, args)
        # mytest(model, queryloader, galleryloader, use_gpu, args)


# Prepare Date

In [4]:
# Data augmentation
args = Args()
conf = Config()
use_gpu = torch.cuda.is_available()
device = 'cuda' if use_gpu else 'cpu'
pin_memory = True if use_gpu else False
temporal_transform_test = TT.TemporalBeginCrop()
dataset = data_manager.init_dataset(name=args.dataset, root=args.root)
spatial_transform_test = ST.Compose([
            ST.Scale((args.height, args.width), interpolation=3),
            ST.ToTensor(),
            ST.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

# return queryloader, galleryloader


=> MARS loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # tracklets
  ------------------------------
  train    |   625 |     8298
  query    |   626 |     1980
  gallery  |   622 |     9330
  ------------------------------
  total    |  1247 |    19608
  number of images per tracklet: 2 ~ 920, average 59.5
  ------------------------------


# Base model Feature Extraction

In [5]:
spatial_transform_train = conf.get_spatial_transform_train(args)
temporal_transform_train = TT.TemporalRandomCrop()

trainloader = DataLoader(
    VideoDataset(dataset.train, spatial_transform=spatial_transform_train, temporal_transform=temporal_transform_test),
    batch_size=32, num_workers=args.workers,
    pin_memory=pin_memory, drop_last=True)


conf.use_linear_to_get_important_features = False
conf.print_model_parameters_trainable = False
conf.use_linear_to_merge_features = False
conf.use_hist = False
args.pretrain = 'logs/row41/best_model.pth.tar'
conf.print_model_layers = False

print("Initializing model: {}".format(args.arch))
seed_everythings()
model = models.init_model(name=args.arch, conf=conf, num_classes=dataset.num_train_pids)
print("Model size: {:.5f}M".format(sum(p.numel() for p in model.parameters())/1000000.0))
model = model.to(device)
model = model.eval()
modify_model(model, args, conf)
with torch.no_grad():

    # test using 4 frames
    since = time.time()
    model.eval()

    tf, t_pids, t_camids = [], [], []
    for batch_idx, (vids, pids, camids) in tqdm(enumerate(trainloader)):
        if use_gpu:
            vids = vids.cuda()

        feat = model(vids)
        feat = feat.mean(1)
        feat = model.bn(feat)
        feat = feat.data.cpu()

        tf.append(feat)
        t_pids.extend(pids)
        t_camids.extend(camids)
        
    tf = torch.cat(tf, 0)
    t_pids = np.asarray(t_pids)
    t_camids = np.asarray(t_camids)
    print("Extracted features for query set, obtained {} matrix".format(tf.shape))

/mnt/1tra/shamgholi/miniconda3/envs/reid/lib/python3.7/site-packages/torchvision/transforms/transforms.py:279: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


Initializing model: ap3dres50
Model size: 11.75930M
pretrain state dict loaded
----------
Model size: 11.75930M


259it [01:19,  3.27it/s]

Extracted features for query set, obtained torch.Size([8288, 512]) matrix


In [13]:
# from collections import Counter

# res = Counter(t_pids) # equals to list(set(words))
# # for k,v in list(zip(res.keys(), res.values()))[:5]:
# #     print(k, v)
# # res.most_common(10)
# [(l,k) for k,l in sorted([(j,i) for i,j in res.items()], reverse=False)][:5]

[(8, 1), (5, 2), (6, 2), (119, 2), (251, 2)]

# Train logistic regression

In [17]:
logistic_dataset = []
logistic_labels = []
t_qids_unique = np.unique(t_pids)
for i, tid in tqdm(enumerate(t_pids)):
    population = [ind for ind, pid in enumerate(t_pids) if pid == tid]
    population.remove(i)
    # if len(population) > 3:
    #     k = 4
    # elif len(population) > 2:
    #     k = 3
    # elif len(population) > 1:
    #     k = 2
    # else:
    #     k = 1
    try:
        i_sample = random.sample(population=population, k=1)
    except ValueError:
        continue
    feats = tf[i_sample]
    for feat in feats:
        logistic_dataset.append(abs(feat - tf[i]).unsqueeze(0))
        logistic_labels.append(1.0)

        
for i, tid in tqdm(enumerate(t_pids)):
    population = [ind for ind, pid in enumerate(t_pids) if pid != tid]
    # if len(population) > 3:
    #     k = 4
    # elif len(population) > 2:
    #     k = 3
    # elif len(population) > 1:
    #     k = 2
    # else:
    #     k = 1
    try:
        i_sample = random.sample(population=population, k=1)
    except ValueError:
        continue
    feats = tf[i_sample]
    for feat in feats:
        logistic_dataset.append(abs(feat - tf[i]).unsqueeze(0))
        logistic_labels.append(0.0)
      
    
    
logistic_dataset = torch.cat(logistic_dataset, 0).numpy()
logistic_labels = torch.FloatTensor(logistic_labels).numpy()
print(logistic_dataset.shape)



from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.utils import shuffle
logistic_dataset_shuff, logistic_labels_shuff = shuffle(logistic_dataset, logistic_labels)

x_train, y_train = logistic_dataset_shuff[2000:], logistic_labels_shuff[2000:]
x_val, y_val = logistic_dataset_shuff[:2000], logistic_labels_shuff[:2000]

In [22]:
%%time
clf = svm.SVC(probability=True)
# clf = LogisticRegression(random_state=1)
clf.fit(x_train, y_train)
clf.score(x_val, y_val)

0.9975

# prepare test data

In [9]:
import random
from scipy.io import loadmat

# random.shuffle(dataset.query)
# query_train = dataset.query[:900]
# query_test = dataset.query[900:]

queryloader = DataLoader(
    VideoDataset(dataset.query, spatial_transform=spatial_transform_test, temporal_transform=temporal_transform_test),
    batch_size=args.test_batch, shuffle=False, num_workers=0,
    pin_memory=pin_memory, drop_last=False)
galleryloader = DataLoader(
    VideoDataset(dataset.gallery, spatial_transform=spatial_transform_test, temporal_transform=temporal_transform_test),
    batch_size=args.test_batch, shuffle=False, num_workers=0,
    pin_memory=pin_memory, drop_last=False)

# test set feature extraction

In [37]:
def extract_test_feature(model, queryloader, galleryloader, use_bn=True):

    with torch.no_grad():
        # test using 4 frames
        since = time.time()
        model.eval()

        qf, q_pids, q_camids = [], [], []
        for batch_idx, (vids, pids, camids) in enumerate(queryloader):
            if use_gpu:
                vids = vids.cuda()

            feat = model(vids)
            feat = feat.mean(1)
            if use_bn:
                feat = model.bn(feat)
            feat = feat.data.cpu()

            qf.append(feat)
            q_pids.extend(pids)
            q_camids.extend(camids)

        qf = torch.cat(qf, 0)
        q_pids = np.asarray(q_pids)
        q_camids = np.asarray(q_camids)
        print("Extracted features for query set, obtained {} matrix".format(qf.shape))

        gf, g_pids, g_camids = [], [], []
        for batch_idx, (vids, pids, camids) in enumerate(galleryloader):
            if use_gpu:
                vids = vids.cuda()

            feat = model(vids)
            feat = feat.mean(1)
            if use_bn:
                feat = model.bn(feat)
            feat = feat.data.cpu()

            gf.append(feat)
            g_pids.extend(pids)
            g_camids.extend(camids)

        gf = torch.cat(gf, 0)
        g_pids = np.asarray(g_pids)
        g_camids = np.asarray(g_camids)

        if args.dataset == 'mars':
            # gallery set must contain query set, otherwise 140 query imgs will not have ground truth.
            gf = torch.cat((qf, gf), 0)
            g_pids = np.append(q_pids, g_pids)
            g_camids = np.append(q_camids, g_camids)

        print("Extracted features for gallery set, obtained {} matrix".format(gf.shape))

        time_elapsed = time.time() - since
        print('Extracting features complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    return qf, q_pids, q_camids, gf, g_pids, g_camids

In [11]:
# query_train_loader = DataLoader(
#     VideoDataset(query_train, spatial_transform=spatial_transform_test, temporal_transform=temporal_transform_test),
#     batch_size=args.test_batch, shuffle=False, num_workers=0,
#     pin_memory=pin_memory, drop_last=False)

conf.use_linear_to_get_important_features = False
conf.print_model_parameters_trainable = False
conf.use_linear_to_merge_features = False
conf.use_hist = False
args.pretrain = 'logs/row41/best_model.pth.tar'
conf.print_model_layers = False

print("Initializing model: {}".format(args.arch))
seed_everythings()
model = models.init_model(name=args.arch, conf=conf, num_classes=dataset.num_train_pids)
print("Model size: {:.5f}M".format(sum(p.numel() for p in model.parameters())/1000000.0))
model = model.to(device)
model = model.eval()
modify_model(model, args, conf)

In [ ]:
qf, q_pids, q_camids, gf, g_pids, g_camids = extract_test_feature(model, queryloader, galleryloader)

## base model test

In [12]:
print("Computing distance matrix")
m, n = qf.size(0), gf.size(0)
distmat = torch.zeros((m,n))

if args.distance == 'euclidean':
    distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
              torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
    for i in range(m):
        distmat[i:i+1].addmm_(1, -2, qf[i:i+1], gf.t())
else:
    q_norm = torch.norm(qf, p=2, dim=1, keepdim=True)
    g_norm = torch.norm(gf, p=2, dim=1, keepdim=True)
    qf = qf.div(q_norm.expand_as(qf))
    gf = gf.div(g_norm.expand_as(gf))
    for i in range(m):
        distmat[i] = - torch.mm(qf[i:i+1], gf.t())
distmat = distmat.numpy()

print("Computing CMC and mAP")
cmc, mAP = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)

print("Results ----------")
print('top1:{:.1%} top5:{:.1%} top10:{:.1%} mAP:{:.1%}'.format(cmc[0],cmc[4],cmc[9],mAP))
print("------------------")

Computing distance matrix
Computing CMC and mAP
Results ----------
top1:83.5% top5:93.1% top10:95.1% mAP:73.2%
------------------


# run logistic regression on test set

In [24]:
from tqdm.auto import tqdm

print("Computing distance matrix")
m, n = qf.size(0), gf.size(0)
distmat = np.zeros((m,n))

for iq, q in tqdm(enumerate(qf)):
    dif = abs(gf - q)
    sim = clf.predict_proba(dif)[:, 1]
    distmat[iq, :] =  -sim


# distmat = distmat.numpy()
print(distmat.shape)

print("Computing CMC and mAP")
cmc, mAP = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)

print("Results ----------")
print('top1:{:.1%} top5:{:.1%} top10:{:.1%} mAP:{:.1%}'.format(cmc[0],cmc[4],cmc[9],mAP))
print("------------------")

Computing distance matrix


1980it [1:02:58,  1.91s/it]


(1980, 11310)
Computing CMC and mAP
Results ----------
top1:0.2% top5:0.4% top10:0.4% mAP:0.5%
------------------


# compute hist distance

In [2]:
import torch
import numpy as np
import torch.nn.functional as F

def hist_intersection(hist_1, hist_2):
    if hist_1.ndim == 1:
        hist_1 = hist_1.view(1, hist_1.shape[0])
    if hist_2.ndim == 1:
        hist_2 = hist_2.view(1, hist_2.shape[0])
        
    minima = torch.minimum(hist_1, hist_2)
    intersection = torch.true_divide(torch.sum(minima, dim=1), torch.sum(torch.maximum(hist_1, hist_2), dim=1))
    return intersection

/mnt/1tra/shamgholi/miniconda3/envs/reid/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print("Computing distance matrix")
m, n = qf.size(0), gf.size(0)
distmat = np.zeros((m,n))

for iq, q in tqdm(enumerate(qf)):
    for ig, g in enumerate(gf[iq:]):
        d = hist_intersection(q, g)
        distmat[iq, ig] = d
        distmat[ig, iq] = d

In [8]:
v1 = torch.FloatTensor([[1,10,1,1,1,1,1,1]])
v2 = torch.FloatTensor([[1,5,1,1,1,1,1,1]])
# v2 = v2.repeat(len(v1), 1)

print(hist_intersection(v1, v2))
print(hist_intersection(v2, v1))
torch.nn.CosineSimilarity()(v1, v2)

tensor([0.7059])
tensor([0.7059])


tensor([0.9741])

# 

In [38]:
import numpy as np
conf.use_linear_to_get_important_features = False
conf.print_model_parameters_trainable = False
conf.use_linear_to_merge_features = False
conf.use_hist = True
conf.centers = [0.1, 0.3, 0.5, 0.7, 0.9, 1.1]
conf.widths = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
conf.init_hist("HistYusufLayer")
args.pretrain = 'logs/row53/best_model.pth.tar'
conf.print_model_layers = False

print("Initializing model: {}".format(args.arch))
seed_everythings()
model = models.init_model(name=args.arch, conf=conf, num_classes=dataset.num_train_pids)
print("Model size: {:.5f}M".format(sum(p.numel() for p in model.parameters())/1000000.0))
model = model.to(device)
model = model.eval()
modify_model(model, args, conf)
print('model loaded ...')

qf1, q_pids1, q_camids1, gf1, g_pids1, g_camids1 = extract_test_feature(model, queryloader, galleryloader, use_bn=False)

Initializing model: ap3dres50
Model size: 13.37671M
pretrain state dict loaded
----------
Model size: 13.37671M
model loaded ...
Extracted features for query set, obtained torch.Size([1980, 3072]) matrix
Extracted features for gallery set, obtained torch.Size([11310, 3072]) matrix
Extracting features complete in 1m 44s


In [39]:
from tqdm.auto import tqdm
print("Computing distance matrix")
m, n = qf1.size(0), gf1.size(0)
distmat = np.zeros((m,n))

for iq, q in tqdm(enumerate(qf1)):
    q_repeat = q.repeat(len(gf1), 1)
    d = 1 - hist_intersection(q_repeat, gf1)
    distmat[iq, :] = d
    

print("Computing CMC and mAP")
cmc, mAP = evaluate(distmat, q_pids, g_pids, q_camids, g_camids)
print("Results ----------")
print('top1:{:.1%} top5:{:.1%} top10:{:.1%} mAP:{:.1%}'.format(cmc[0],cmc[4],cmc[9],mAP))
print("------------------")

Computing distance matrix
Computing CMC and mAP
Results ----------
top1:69.9% top5:84.0% top10:87.8% mAP:52.4%
------------------


In [43]:
%%time
import numpy as np
conf.use_linear_to_get_important_features = False
conf.print_model_parameters_trainable = False
conf.use_linear_to_merge_features = False
conf.use_hist = True
#conf.centers = [0.1, 0.3, 0.5, 0.7, 0.9, 1.1]
conf.centers = [0.5, 1.25, 1.75,  2.2, 2.6, 3.0]
#conf.widths = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
conf.widths = [0.5, 0.25, 0.25, 0.2, 0.2, 0.2]
conf.init_hist("HistYusufLayer")
args.pretrain = 'logs/row53/best_model.pth.tar'
conf.print_model_layers = False

print("Initializing model: {}".format(args.arch))
seed_everythings()
model = models.init_model(name=args.arch, conf=conf, num_classes=dataset.num_train_pids)
print("Model size: {:.5f}M".format(sum(p.numel() for p in model.parameters())/1000000.0))
model = model.to(device)
model = model.eval()
modify_model(model, args, conf)
print('model loaded ...')

qf1, q_pids1, q_camids1, gf1, g_pids1, g_camids1 = extract_test_feature(model, queryloader, galleryloader, use_bn=False)

from tqdm.auto import tqdm
print("Computing distance matrix")
m, n = qf1.size(0), gf1.size(0)
distmat = np.zeros((m,n))

for iq, q in tqdm(enumerate(qf1)):
    q_repeat = q.repeat(len(gf1), 1)
    d = 1 - hist_intersection(q_repeat, gf1)
    distmat[iq, :] = d
    

print("Computing CMC and mAP")
cmc, mAP = evaluate(distmat, q_pids1, g_pids1, q_camids1, g_camids1)
print("Results ----------")
print('top1:{:.1%} top5:{:.1%} top10:{:.1%} mAP:{:.1%}'.format(cmc[0],cmc[4],cmc[9],mAP))
print("------------------")

Initializing model: ap3dres50
Model size: 13.37671M
pretrain state dict loaded
----------
Model size: 13.37671M
model loaded ...
Extracted features for query set, obtained torch.Size([1980, 3072]) matrix
Extracted features for gallery set, obtained torch.Size([11310, 3072]) matrix
Extracting features complete in 1m 42s
Computing distance matrix


1980it [02:02, 16.23it/s]


Computing CMC and mAP
Results ----------
top1:63.2% top5:74.2% top10:76.9% mAP:38.1%
------------------
CPU times: user 15min 29s, sys: 5min 31s, total: 21min 1s
Wall time: 3min 50s
